In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-07-07 04:33:41--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2024-07-07 04:33:41 (6.83 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [5]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

In [6]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

unzip_data('10_food_classes_10_percent.zip')

--2024-07-07 04:37:52--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.114.207, 142.250.115.207, 142.251.116.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.114.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M  18.9MB/s    in 8.5s    

2024-07-07 04:38:01 (19.0 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [8]:
walk_through_dir('10_food_classes_10_percent')

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent

In [9]:
train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

In [11]:
import tensorflow as tf

IMG_SIZE = (224,224)
BATCH_SIZE = 32

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=IMG_SIZE,
                                                                 label_mode='categorical',
                                                                 batch_size=BATCH_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                 image_size=IMG_SIZE,
                                                                 label_mode='categorical',
                                                                 batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [13]:
train_data.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [16]:
for images, labels in train_data.take(1):
    print(images, labels)

tf.Tensor(
[[[[230.50128    214.50128    241.50128   ]
   [226.00032    212.00032    238.00032   ]
   [231.40625    217.40625    243.40625   ]
   ...
   [211.71591    206.18918    239.95255   ]
   [213.51756    209.51756    242.51756   ]
   [213.79814    209.79814    242.79814   ]]

  [[228.67761    214.67761    240.67761   ]
   [230.1113     216.1113     242.1113    ]
   [234.8243     220.8243     246.8243    ]
   ...
   [216.61668    212.61668    245.61668   ]
   [215.264      211.264      244.264     ]
   [213.47447    209.47447    242.47447   ]]

  [[228.61798    214.4037     241.04655   ]
   [226.12213    211.90785    238.5507    ]
   [225.1859     210.37181    238.8141    ]
   ...
   [213.73062    209.73062    242.73062   ]
   [218.27939    214.27939    247.27939   ]
   [218.92409    215.56694    248.35266   ]]

  ...

  [[ 21.571472    10.571472    44.571472  ]
   [ 21.583908    10.583907    44.583908  ]
   [ 22.385561    11.385562    45.385563  ]
   ...
   [ 13.           4.836

## Model 0

In [18]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)

base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(224,224,3), name='input')

# If using a model like ResNet50V2 you will need to normalize
# x = tf.keras.experimental.preprocessing.Rescaling(1./255)(inputs)

x = base_model(inputs)
print(f'Shape after passing inputs through base model: {x.shape}')

x = tf.keras.layers.GlobalAveragePooling2D(name='global_average_pool')(x)
print(f'Shape after GlobalAveragePooling2D: {x.shape}')

output = tf.keras.layers.Dense(10, activation='softmax', name='output_layer')(x)

model_0 = tf.keras.Model(inputs, output)

Shape after passing inputs through base model: (None, 7, 7, 1280)
Shape after GlobalAveragePooling2D: (None, 1280)


In [20]:
model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

history0 = model_0.fit(train_data,
                       epochs=5,
                       steps_per_epoch=len(train_data),
                       validation_data=test_data,
                       validation_steps=len(test_data),
                       callbacks=[create_tensorboard_callback(dir_name='tensorflow_hub',
                                                              experiment_name='model_0_efficient_net')])

Saving TensorBoard log files to: tensorflow_hub/model_0_efficient_net/20240707-050950
Epoch 1/5


2024-07-07 05:09:55.577123: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2024-07-07 05:09:57.402022: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f41712b1fa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-07 05:09:57.402058: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-07-07 05:09:57.414231: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1720328997.506107    8285 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


24/24 [==============================] - 12s 228ms/step - loss: 1.9291 - accuracy: 0.3893 - val_loss: 1.3593 - val_accuracy: 0.7256
Epoch 2/5
24/24 [==============================] - 3s 142ms/step - loss: 1.1578 - accuracy: 0.7333 - val_loss: 0.9076 - val_accuracy: 0.8168
Epoch 3/5
24/24 [==============================] - 3s 137ms/step - loss: 0.8343 - accuracy: 0.8307 - val_loss: 0.7232 - val_accuracy: 0.8420
Epoch 4/5
24/24 [==============================] - 3s 132ms/step - loss: 0.6848 - accuracy: 0.8520 - val_loss: 0.6270 - val_accuracy: 0.8532
Epoch 5/5
24/24 [==============================] - 3s 136ms/step - loss: 0.5929 - accuracy: 0.8560 - val_loss: 0.5688 - val_accuracy: 0.8600


In [23]:
base_model.summary()

Model: "efficientnetv2-b0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 rescaling_1 (Rescaling)     (None, None, None, 3)        0         ['input_2[0][0]']             
                                                                                                  
 normalization_1 (Normaliza  (None, None, None, 3)        0         ['rescaling_1[0][0]']         
 tion)                                                                                            
                                                                                                  
 stem_conv (Conv2D)          (None, None, None, 32)       864       ['normalizatio

In [24]:
model_0.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Functio  (None, None, None, 1280   5919312   
 nal)                        )                                   
                                                                 
 global_average_pool (Globa  (None, 1280)              0         
 lAveragePooling2D)                                              
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 5932122 (22.63 MB)
Trainable params: 12810 (50.04 KB)
Non-trainable params: 5919312 (22.58 MB)
_________________________________________________________________
